In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.layers import Layer, Dense, Flatten, Reshape, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.backend import random_normal
from keras.metrics import Mean
from keras import backend as K
from typing import List

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

### Data

In [3]:
df_data = pd.read_pickle("D:/Dev/Tutorial/Tensorflow/data/2135_기동_RCP_1A_200926_201005.pkl").astype('float32')

In [5]:
df_data.index = pd.to_datetime(df_data.index)

In [ ]:
fig, axs = plt.subplots(10, figsize=(30,60))
for ax, (colname, coldata) in zip(axs, df_data.iteritems()):
    ax.plot(coldata)

In [8]:
batch_size = 1024
window_size = 8

In [9]:
from preprocessing.scaler import StandardScaler

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(df_data)
data_scaled = scaler.transform(df_data)

In [12]:
tf_data = tf.data.Dataset.from_tensor_slices(data_scaled)
tf_data

<TensorSliceDataset element_spec=TensorSpec(shape=(10,), dtype=tf.float32, name=None)>

In [13]:
tf_data_windowed = tf_data.window(8, shift=1, drop_remainder=True)
tf_data_windowed

<WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(10,), dtype=tf.float32, name=None), TensorShape([]))>

In [14]:
tf_data_windowed_flat = tf_data_windowed.flat_map(lambda window: window.batch(batch_size=window_size, drop_remainder=True)).batch(batch_size, drop_remainder=True)
dataset = tf_data_windowed_flat.shuffle(buffer_size = len(tf_data_windowed))

In [15]:
dataset_train = dataset.enumerate().filter(lambda f, data:f%10 >= 2).map(lambda f, data: data).prefetch(1).cache()
dataset_valid = dataset.enumerate().filter(lambda f, data:f%10 < 2).map(lambda f, data: data).prefetch(1).cache()

Cause: could not parse the source code of <function <lambda> at 0x0000020362BFAF80>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda f, data: data

Match 1:
lambda f, data: f % 10 >= 2

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x0000020362BFAF80>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda f, data: data

Match 1:
lambda f, data: f % 10 >= 2

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x0000020307AFF7F0>: found multiple defi

In [16]:
class Sampling(Layer):

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [17]:

mse_loss_fn = tf.keras.losses.MeanSquaredError()


class VAE(Model):
    def __init__(self, window_size:int, feat_dim:int, latent_dim:int, hidden_layer_sizes:List,kl_weight:float=0.0001, activation:str="relu", **kwargs):
        super(VAE, self).__init__(**kwargs)
        
        self.window_size = window_size
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.kl_weight = kl_weight

        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.encoder = self._get_encoder()
        self.decoder = self._get_decoder()
        
        self.metric_total_loss = Mean(name="total_loss")
        self.metric_recons_loss = Mean(name="recons_loss")
        self.metric_kl_loss = Mean(name="kl_loss")
        
        self.optimizer = Adam(learning_rate=0.0001)

    def _get_encoder(self):
        self.encoder_inputs = Input(
            shape=(self.window_size, self.feat_dim), name="encoder_input"
        )

        x = Flatten()(self.encoder_inputs)
        for i, M_out in enumerate(self.hidden_layer_sizes):
            x = Dense(M_out, activation=self.activation, name=f"enc_dense_{i}")(x)

        z_mean = Dense(self.latent_dim, name="z_mean")(x)
        z_log_var = Dense(self.latent_dim, name="z_log_var")(x)
        encoder_output = Sampling()([z_mean, z_log_var])
        self.encoder_output = encoder_output

        encoder = Model(
            self.encoder_inputs, [z_mean, z_log_var, encoder_output], name="encoder"
        )
        return encoder

    def _get_decoder(self):
        decoder_inputs = Input(shape=(self.latent_dim), name="decoder_input")

        x = decoder_inputs
        for i, M_out in enumerate(reversed(self.hidden_layer_sizes)):
            x = Dense(M_out, activation=self.activation, name=f"dec_dense_{i}")(x)
        x = Dense(self.window_size * self.feat_dim, name="decoder_final_dense")(x)
        self.decoder_outputs = Reshape(target_shape=(self.window_size, self.feat_dim))(x)
        decoder = Model(decoder_inputs, self.decoder_outputs, name="decoder")
        return decoder

    def call(self, X):
        z_mean, _, _ = self.encoder(X)
        x_decoded = self.decoder(z_mean)
        return x_decoded

    def _get_reconstruction_loss(self, X, X_recons):
        return K.mean(mse_loss_fn(X, X_recons))

    def _get_kl_loss(self, z_mean, z_log_var):
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.mean(kl_loss) + 1e-8
        kl_loss *= -0.5
        return self.kl_weight * kl_loss
    
    def loss_function(self, X, X_recons, z_mean, z_log_var):
        reconstruction_loss = self._get_reconstruction_loss(X, X_recons)
        kl_loss = self._get_kl_loss(z_mean, z_log_var)
        total_loss = reconstruction_loss + kl_loss
        return total_loss, reconstruction_loss, kl_loss
    
    @tf.function
    def train_step(self, X) -> dict:
        with tf.GradientTape() as tape:
            # * Inference
            z_mean, z_log_var, z = self.encoder(X)
            reconstruction = self.decoder(z)

            # * Calculate Loss
            total_loss, reconstruction_loss, kl_loss = self.loss_function(
                X, reconstruction, z_mean, z_log_var
            )

        # * Backprop
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # * Update training Metrics
        self.metric_total_loss.update_state(total_loss)
        self.metric_recons_loss.update_state(reconstruction_loss)
        self.metric_kl_loss.update_state(kl_loss)

        return {metric.name: metric.result() for metric in self.metrics}

    @tf.function
    def test_step(self, X) -> dict:
        z_mean, z_log_var, z = self.encoder(X)
        reconstruction = self.decoder(z_mean)

        # * Calculate Loss
        total_loss, reconstruction_loss, kl_loss = self.loss_function(
            X, reconstruction, z_mean, z_log_var
        )

        self.metric_total_loss.update_state(total_loss)
        self.metric_recons_loss.update_state(reconstruction_loss)
        self.metric_kl_loss.update_state(kl_loss)

        return {metric.name: metric.result() for metric in self.metrics}

    def build_graph(self):
        x = tf.keras.layers.Input(shape=(self.window_size, self.feat_dim))
        return Model(inputs=[x], outputs=self.call(x))


In [18]:
model = VAE(window_size=window_size, feat_dim=10, latent_dim=16, hidden_layer_sizes=[64,32,16],)

In [19]:
import pickle

In [22]:
model.compile()
with open("model.obj", "wb") as f:
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: ram://6d6f814d-3d92-4571-bd3e-9a02f5996cd5/assets
INFO:tensorflow:Assets written to: ram://a5c07208-0823-415c-b18f-2d237ca23ebb/assets


TypeError: cannot pickle 'weakref.ReferenceType' object

In [87]:
with open("model.obj", "rb") as f:
    loaded_obj = pickle.load(f)

AttributeError: 'Mean' object has no attribute 'update_state_fn'

In [76]:
pickle.dump(model, "model.obj")

TypeError: file must have a 'write' attribute

In [23]:
v = np.zeros([1,8,10])

In [24]:
model(v)

<tf.Tensor: shape=(1, 8, 10), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

In [65]:
keras.models.save_model(model, "saved.tf")

INFO:tensorflow:Assets written to: saved.tf\assets


INFO:tensorflow:Assets written to: saved.tf\assets


In [66]:
loaded = keras.models.load_model("saved.tf")

In [73]:
x = tf.keras.layers.Input(shape=(window_size,10))
model__ = keras.Model(inputs=x, outputs=loaded(x))
tf.keras.utils.plot_model(model__, show_shapes=True, show_layer_names=True, expand_nested=True)

In [71]:
loaded(x)

<KerasTensor: shape=(None, 8, 10) dtype=float32 (created by layer 'vae_4')>

In [67]:
loaded.build_graph()

AttributeError: 'VAE' object has no attribute 'build_graph'

In [90]:
tf.keras.utils.plot_model(model.build_graph(), show_shapes=True, show_layer_names=True, expand_nested=True, to_file="mymodel_model.png")

In [49]:
tf.keras.utils.plot_model(model.build_graph(), show_shapes=True, show_layer_names=True, expand_nested=True)

TypeError: 'NoneType' object is not callable

In [ ]:
tf.keras.layers.Input(shape=(8, 10))

In [54]:
model.build_graph().summary(expand_nested=True)

TypeError: To be compatible with tf.function, Python functions must return zero or more Tensors or ExtensionTypes or None values; in compilation of <function VAE.build_graph at 0x000002635F2137F0>, found return value of type Functional, which is not a Tensor or ExtensionType.

In [25]:
optimizer = Adam()

In [26]:
def train_model(model:object, dataset, epochs:int=10):
    for epoch in range(epochs):
        print(f"Epoch: {epoch}")
        
        for batch_data in dataset:
            
            model.train_step(batch_data)
            
        _metrics_result = [
                    (metric.name, metric.result().numpy()) for metric in model.metrics
                ]
        print(_metrics_result)
        for metric in model.metrics:
            metric.reset_states()
    

In [27]:
train_model(model, dataset_train, 2)

Epoch: 0


KeyboardInterrupt: 

In [28]:
model.compile(optimizer)

In [29]:
model.fit(dataset_train, epochs=10, validation_data=dataset_valid)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
class GaonTrainer:
    
    @classmethod
    def custom_fit(cls, model, epoch, *args, **kwargs):
        model.fit(*args, **kwargs)

In [ ]:
GaonTrainer.custom_fit(model, dddd)